In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import multiprocessing
from sklearn.metrics import roc_auc_score

# Prepare OOF

CV 0.867 OOF was taken from @hidehisaarai1213 's notebook.

https://www.kaggle.com/hidehisaarai1213/g2net-tf-on-the-fly-cqt-tpu-training

In [ ]:
def calc_metric(df):
    return roc_auc_score(df["y_true"], df["y_pred"])

In [ ]:
oof = pd.read_csv("../input/g2net-tf-on-the-fly-cqt-tpu-training/oof/oof.csv")
print(f"CV score is {calc_metric(oof):.3f}")

# Leaderboard simulation
We have 226000 test data, and Public leaderboard is calculated with 16%, Private is on the other 84%.

We assume that the number of each target is eqaul as same as the training datasets distribution.

226000 samples are randomly extracted from 560000 oof so that the ratio of each target is the same. 

This random sampling is performed 10000 times.

Note that there is possibility that the same data in oof can be used for both Public and Private estimation.

In [ ]:
num_data = 226000
num_public = int(0.16 * num_data)
num_private = int(0.84 * num_data)
sampling_times = 10000

In [ ]:
def public_estimation(dummy):
    df = pd.concat([oof[oof["y_true"] == 1].sample(num_public//2), 
                    oof[oof["y_true"] == 0].sample(num_public//2)])

    return calc_metric(df)


def private_estimation(dummy):
    df = pd.concat([oof[oof["y_true"] == 1].sample(num_private//2), 
                    oof[oof["y_true"] == 0].sample(num_private//2)])

    return calc_metric(df)

In [ ]:
processes = multiprocessing.cpu_count()
with multiprocessing.Pool(processes=processes) as pool:
    public_auc = pool.imap_unordered(public_estimation, range(sampling_times))
    public_auc = tqdm(public_auc, total=sampling_times)
    public_auc = list(public_auc)

In [ ]:
processes = multiprocessing.cpu_count()
with multiprocessing.Pool(processes=processes) as pool:
    private_auc = pool.imap_unordered(private_estimation, range(sampling_times))
    private_auc = tqdm(private_auc, total=sampling_times)
    private_auc = list(private_auc)

In [ ]:
plt.figure(figsize=(12, 8))
sns.distplot(public_auc, color="red")
sns.distplot(private_auc, color="blue")
plt.legend(labels=["Public", "Private"], fontsize=12)

# Conclusions
If you have stable relationship between CV and LB, it is estimated that...
- public LB is CV ± 0.005.
- private LB is CV ± 0.002.
- 0.007 shake up/down is possible.

In this notebook, I proposed a very simple shake estimation.

Using multiple oof and calculating std of difference between public and private may be better way of another simulation.

Any opinions are welcome!